In [1]:
# FILE:   WBC_imgSeg.R
# AUTH:   P.Compeau, N. Matamala
# STRT:   04.23.2019; 11:35 A
# UPDT:   05.16.2019; 01:30 P

# DISC:   Majority of this code comes from segmentationNuc.R written by 
#         P. Compeau with modifications by N. Matamala

In [2]:
# Required Libraries
library("EBImage")
library("jpeg")
library("tiff")

In [3]:
# dir Counter
i = 1

# Set path for raw image files
path="~/Desktop/WBC_PCAPipeline/Data/"
rawImgs=paste(path, "RawImgs/", sep="")

# Set up directory path for segemented images
segImgs=paste(path, "SegImgs_", sep="")
bwImgs=paste(path, "BWImgs_", sep="")

# Check if unique seg directory exists, otherwise create one
while (file.exists(paste(segImgs, toString(i), sep=""))) {
  i = i + 1
}
print(noquote(paste("Creating", paste(segImgs, toString(i), sep=""), "directory for segmented images.")))
dir.create(paste(segImgs, toString(i), sep=""), showWarnings = FALSE);
print(noquote(paste("Creating", paste(bwImgs, toString(i), sep=""), "directory for binarized images.")))
dir.create(paste(bwImgs, toString(i), sep=""), showWarnings = FALSE)
outDir=paste(segImgs, toString(i), sep="")
setwd(rawImgs)

[1] Creating ~/Desktop/WBC_PCAPipeline/Data/SegImgs_2 directory for segmented images.
[1] Creating ~/Desktop/WBC_PCAPipeline/Data/BWImgs_2 directory for binarized segmented images.


In [4]:
# Gather all files within the directory above
all.files <- list.files()
my.files <- grep("*.jpg", all.files, value=T)

In [5]:
print(noquote("Starting nucleus segmentation..."))

# Loop through each file and process each image individually
for (i in my.files) {
  print(noquote(paste("Segmenting nucleus from file", i)))
  # Read the image, change to its directory
  nuc = readImage(paste(rawImgs, i, sep=""))
  
  # Gather dimensions of the image
  wid = dim(nuc)[1]
  hgt = dim(nuc)[2]
    
  # Each nuclear stain has low red, low green and high blue.
  
  # Need to invert the red and green channels, and then threshold according to
  # the above criteria
  nuc_r = channel(nuc, 'r')
  nuc_g = channel(nuc, 'g')
  nuc_b = channel(nuc, 'b')
  
  # Assigned thresholds for low red, low green, and high blue.
  r_threshold = 0.65
  g_threshold = 0.60
  b_threshold = 0.5975
  
  # Apply the thresholds accordingly
  nuc_rTH = nuc_r < r_threshold
  nuc_gTH = nuc_g < g_threshold
  nuc_bTH = nuc_b > b_threshold
  
  nucleusComp = nuc_rTH & nuc_gTH & nuc_bTH
  nucleusBW = bwlabel(fillHull(nucleusComp))
  
  # Compute features for objects that have made the threshold boundaries
  features = computeFeatures.shape(nucleusBW)
  
  # Assigned area threshold
  area_threshold <- 1500
  
  # Find all features that do not meet the area threshold
  indices = which(features < area_threshold)
  nucleusFin = rmObjects(nucleusBW, indices)
  
  newFeatures = computeFeatures.shape(nucleusFin)
  
  # Write final nucleus image to disk
  filename = paste(outDir, (paste("/segNuc_", i, sep="")), sep="")
  writeImage(nucleusFin, filename)
}
print(noquote("DONE!"))

[1] Starting nucleus segmentation...
[1] Segmenting nucleus from file BloodImage_00001.jpg
[1] Segmenting nucleus from file BloodImage_00002.jpg
[1] Segmenting nucleus from file BloodImage_00003.jpg
[1] Segmenting nucleus from file BloodImage_00004.jpg
[1] Segmenting nucleus from file BloodImage_00005.jpg
[1] Segmenting nucleus from file BloodImage_00006.jpg
[1] Segmenting nucleus from file BloodImage_00007.jpg
[1] Segmenting nucleus from file BloodImage_00008.jpg
[1] Segmenting nucleus from file BloodImage_00009.jpg
[1] Segmenting nucleus from file BloodImage_00011.jpg
[1] Segmenting nucleus from file BloodImage_00012.jpg
[1] Segmenting nucleus from file BloodImage_00013.jpg
[1] Segmenting nucleus from file BloodImage_00014.jpg
[1] Segmenting nucleus from file BloodImage_00015.jpg
[1] Segmenting nucleus from file BloodImage_00016.jpg
[1] Segmenting nucleus from file BloodImage_00017.jpg
[1] Segmenting nucleus from file BloodImage_00018.jpg
[1] Segmenting nucleus from file BloodImage_0

[1] DONE!
